In [120]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/dataset91/data.py", dst = "../working/data.py")
copyfile(src = "../input/dataset91/utils.py", dst = "../working/utils.py")
copyfile(src = "../input/dataset91/training.py", dst = "../working/training.py")
copyfile(src = "../input/dataset91/layers.py", dst = "../working/layers.py")
#copyfile(src = "../input/notes1/weights2.npy", dst = "../working/weights2.npy")
# import all our functions
#from data import load_data

'../working/layers.py'

In [121]:
# Estimate the GPS clusters
print("Estimating clusters...")
#clusters = get_clusters(data.train_labels)
n_epochs=100
batch_size=200
save_prefix='mymodel'
config=3


Estimating clusters...


In [122]:
!pip install pickle5
import pickle5 as pickle
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
from scipy.interpolate import splrep
from IPython.core.display import display_html
from keras.models import load_model
from utils import np_haversine, density_map, get_clusters, plot_embeddings
from data import load_data
from training import start_new_session, process_features, create_model

# Display plots inline
%matplotlib inline

# Fix random seed for reproducibility
np.random.seed(42)

In [123]:
clusters_cache = '/kaggle/input/dataset91/clusters.npy'
clusters=np.load(clusters_cache, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [124]:
file='/kaggle/input/dataset91/train0.npy'
train0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train1.npy'
train1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train2.npy'
train2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train3.npy'
train3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train4.npy'
train4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train5.npy'
train5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/train6.npy'
train6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
train=[train0,train1,train2,train3,train4,train5,train6]

In [125]:
file='/kaggle/input/dataset91/validation0.npy'
validation0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation1.npy'
validation1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation2.npy'
validation2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation3.npy'
validation3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation4.npy'
validation4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation5.npy'
validation5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/dataset91/validation6.npy'
validation6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
validation=[validation0,validation1,validation2,validation3,validation4,validation5,validation6]

In [126]:
!pip3 install pickle5
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)
tl_cache = '/kaggle/input/dataset91/train-labels.npy'
vl_cache = '/kaggle/input/dataset91/validation-labels.npy'
train_labels=np.load(tl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
validation_labels=np.load(vl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [127]:
train=train[6]
validation=validation[6]

In [128]:
def first_last_k(coords):
    """
    Returns a list with the first k and last k GPS coordinates from the given trip.
    The returned list contains 4k values (latitudes and longitudes for 2k points).
    """
    k = 5
    partial = [coords[0] for i in range(2*k)]
    num_coords = len(coords)
    if num_coords < 2*k:
        partial[-num_coords:] = coords
    else:
        partial[:k] = coords[:k]
        partial[-k:] = coords[-k:]
    partial = np.row_stack(partial)
    return np.array(partial).flatten()


def process_features(df):
    """
    Process the features required by our model from the given dataframe.
    Return the features in a list so that they can be merged in our model's input layer.
    """
    # Fetch the first and last GPS coordinates
    coords = np.row_stack(df['POLYLINE'].apply(first_last_k))
    # Standardize latitudes (odd columns) and longitudes (even columns)
    latitudes = coords[:,::2]
    coords[:,::2] = scale(latitudes)
    longitudes = coords[:,1::2]
    coords[:,1::2] = scale(longitudes)
    
    return [
        coords
    ]


from tensorflow.compat.v1.keras.layers import LSTM   # CuDNNLSTM
import sys
sys.path.append('..')  # add parent directory to Python path for layers.py access
from layers import Attention, SelfAttention

#trayektori
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
import numpy as np
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine

from utils import get_clusters
from keras.models import Model

def create_model(metadata,clusters,config):
    from layers import Attention, SelfAttention
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)
    coords = Sequential()
    coords.add(Dense(1, input_dim=20,input_shape=(20,1)))
    
    #np.expand_dims(coords,axis=0)
    mergedOut = coords.output

    lstm_layer = tf.keras.layers.LSTM(200, return_sequences=True,return_state=True)
    if config != 0:
        encoder_output, hidden_state, cell_state = lstm_layer(mergedOut)
        attention_input = [encoder_output, hidden_state]
    else:
        encoder_output = LSTM(units=64)(mergedOut)

    # Optional Attention Mechanisms
    if config == 1:
        encoder_output, attention_weights = SelfAttention(size=64,
                                                      num_hops=10,
                                                      use_penalization=False)(encoder_output)
    elif config == 2:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='global')(attention_input)
        encoder_output = Flatten()(encoder_output)
    elif config == 3:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='local-m',
                                                  window_width=10,
                                                  score_function='general')(attention_input)
        encoder_output = Flatten()(encoder_output)
        
    
    #encoder_output=Dense(1)(encoder_output)
    # Determine cluster probabilities using softmax
    mergedOut=Dense(len(clusters))(encoder_output)
    mergedOut=Activation('softmax')(mergedOut)

    # Final activation layer: calculate the destination as the weighted mean of cluster coordinates
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    mergedOut=Activation(destination)(mergedOut)

    newModel = Model([coords.input],mergedOut)
    #use lists if you want more than one input or output  
    
    # Compile the model
    optimizer = SGD(lr=0.01, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients

    newModel.compile(loss=tf_haversine, optimizer=optimizer)
    
    return newModel

In [129]:
model=create_model(metadata,clusters,config)

In [130]:
e=[]
f=[]
g=[]
for i in range(95):
    f.append(0)
    e.append(0)
    g.append(0)

In [131]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(e)
        if z>len(g):
            a=model.history.history['val_loss'][z-len(g)-1]
            b=model.history.history['loss'][z-len(g)-1]
            c=float(a)-float(b)
            if c<0:
                print("Sebelumnya Underfitting")
            else:
                print("Sebelumnya Overfitting")
        e.append(0)
            

In [132]:
class CustCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(f)
        file_path="../working/Newweights"+str(z)
        weight=model.get_weights()
        np.save(file_path, weight)
        f.append(0)

In [133]:
#file='../input/latlstm91gpslocm/Newweights'+str(len(g)-1)+'.npy'
file='./Newweights'+str(len(g)-1)+'.npy'
b=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
model.set_weights(b)

In [134]:
file='../input/dataset91/train-labels.npy'
train_labels=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
file='../input/dataset91/validation-labels.npy'
validation_labels=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [135]:
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/competition-test.pickle'
with open(metadata_cache, 'rb') as handle:
    competition_test = pickle.load(handle)
competition_test_labels=np.load('/kaggle/input/dataset91/competition-test-labels.npy')

In [ ]:

from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping
 
#callbacks = []
#if save_prefix is not None:
        # Save the model's intermediary weights to disk after each epoch
#    file_path="cache/%s-{epoch:03d}-{val_loss:.4f}.hdf5" % save_prefix
#    checkpoint = ModelCheckpoint(file_path,monitor='val_loss',mode='auto',save_weights_only=True,verbose=0)
    #checkpoint = weight_save(model.get_weights(),b)
#    callbacks.append(checkpoint)
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_weights_only=True, verbose=1))
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto'))
    #model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=[weight_save_callback])
#g.append(0)
#epoch>1

#first epoch
#saver = weight_save(model.get_weights(),b)   
#es = EarlyStopping(monitor='val_loss', patience=0, verbose=1)
arc='LSTMnew91-b200-'+str(len(g))+'.log'
csv_logger = CSVLogger(arc, separator=',', append=False)
#print("Creating model...")
#start_new_session()
#print("Train model...")
history=model.fit(train, train_labels,
        initial_epoch=len(g),epochs=100, batch_size=batch_size,
        validation_data=(validation, validation_labels)
        ,callbacks=[csv_logger,CustCallback()])
t=0
while(t<1):
    t=t

Epoch 96/100


2022-01-23 04:18:53.427270: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


7400/7400 [==============================] - ETA: 0s - loss: 1.3746

2022-01-23 04:20:16.546996: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "11000" } environment { key: "cudnn" value: "8005" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16152002560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


7400/7400 [==============================] - 90s 12ms/step - loss: 1.3746 - val_loss: 1.4099
Epoch 97/100
7400/7400 [==============================] - 87s 12ms/step - loss: 1.3738 - val_loss: 1.4099
Epoch 98/100
7400/7400 [==============================] - 87s 12ms/step - loss: 1.3733 - val_loss: 1.4106
Epoch 99/100
7400/7400 [==============================] - 86s 12ms/step - loss: 1.3728 - val_loss: 1.4087
Epoch 100/100
7400/7400 [==============================] - 87s 12ms/step - loss: 1.3721 - val_loss: 1.4115


In [ ]:
test_predictions = model.predict(process_features(competition_test))
print("Mean Haversine Test:{}".format(np_haversine(test_predictions, competition_test_labels).mean()))

In [ ]:
for i in range(10):
    c=91+i
    model=create_model(metadata,clusters,config)
    file='../input/lb91gps/Newweights'+str(c-1)+'.npy'
    #file='./Newweights'+str(c-1)+'.npy'
    b=np.load(file, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
    model.set_weights(b)
    test_predictions = model.predict(process_features(competition_test))
    print("Mean Haversine Test:{}".format(np_haversine(test_predictions, competition_test_labels).mean()))

In [ ]:
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/train.pickle'
with open(metadata_cache, 'rb') as handle:
    train = pickle.load(handle)
train_labels=np.load('/kaggle/input/dataset91/train-labels.npy')

In [ ]:
import pickle5 as pickle
metadata_cache = '/kaggle/input/dataset91/validation.pickle'
with open(metadata_cache, 'rb') as handle:
    test = pickle.load(handle)
test_labels=np.load('/kaggle/input/dataset91/validation-labels.npy')
test = test.reset_index()
del test['index']

In [ ]:
test_predictions = model.predict(process_features(test))
testhav=np_haversine(test_predictions, test_labels)

In [ ]:
rx=testhav
rx.sort()
testhav=np_haversine(test_predictions, test_labels)

In [ ]:
for i in range(20):
    tesh=np.where(testhav==rx[i])
    temp=test
    print(len(temp['POLYLINE_FULL'][tesh[0][0]]))
    print(len(temp['POLYLINE'][tesh[0][0]]))
    print(tesh)

In [ ]:
target=51141
new_df = test[["POLYLINE_FULL"]].copy()
new_df = new_df.rename(columns={"POLYLINE_FULL": "POLYLINE"})

tambah=0
first='P'
second='P'
for i in range(len(test['POLYLINE'][target])):
    ayam=[]
    kolom=first+str(i)
    new_df[kolom]=new_df["POLYLINE"]
    if i==0:
        new_df[kolom][target]=[new_df["POLYLINE"][target][0]]
    else:
        for j in range(i):
            ayam.append(new_df[kolam][target][j])
        ayam.append(new_df["POLYLINE"][target][i])
        new_df[kolom][target]=ayam
    kolam=second+str(i)
    

nama='P'
test_temp=test
aslin=len(test['POLYLINE'][target])
for i in range(aslin):
    print(i+tambah)
    kolom=nama+str(i+tambah)
    testx=new_df[[kolom]].copy()
    testx = testx.rename(columns={kolom: 'POLYLINE'})
    test_temp['POLYLINE'][target]=testx['POLYLINE'][target]
    prediksin=model.predict(process_features(test_temp))
    np.save(str(target)+'new'+kolom+'.npy', prediksin)